In [23]:
import os
import json
import codecs
import pandas as pd

In [28]:
def getTagList(Tag_Statistic, _getNum):
    Tag_List = [ tag_stis[0] for tag_stis in Tag_Statistic[:_getNum]]
    Tag_Index_Dic = {}
    for i in range(len(Tag_List)):
        Tag_Index_Dic.update({Tag_List[i]:i})
    return Tag_List, Tag_Index_Dic

In [34]:
def vectorizeTag(L_T_List, Tag_Index_Dic, _tagDim):
    for song_index in range(len(L_T_List)):
        Tag_Index_List = [ Tag_Index_Dic[tag] for tag in L_T_List[song_index]["Tags"]]
        Tag_Vector = [ 0 for i in range(_tagDim)]
        for index in Tag_Index_List:
            Tag_Vector[index] = 1
        L_T_List[song_index].update({'VectorizeTag':Tag_Vector})

In [96]:
def VectorizedTrainingProducer(lyric_tag_data_filename, tag_statistic_filename, _tagNum):  
    #readData
    with codecs.open(lyric_tag_data_filename,'r',encoding = 'utf8') as infile:
        L_T_List = json.load(infile)
    with codecs.open(tag_statistic_filename,'r',encoding = 'utf8') as infile:
        Tag_Statistic = json.load(infile)
        #filt tags
    Tag_List, Tag_Index_Dic = getTagList(Tag_Statistic, _tagNum)
    
    for song_index in range(len(L_T_List)):
        Tag_Index_List = []
        for tag in L_T_List[song_index]['Tags']:
            if tag not in Tag_List:
                L_T_List[song_index]["Tags"].remove(tag)
            else:
                Tag_Index_List.append(Tag_Index_Dic[tag])
                #print('add')
    #vectorize
        Tag_Vector = [ 0 for i in range(_tagNum)]
        for index in Tag_Index_List:
            Tag_Vector[index] = 1
            #print('change')
        L_T_List[song_index].update({'Vectorized_Tags':Tag_Vector})
    #form Trainging Data
    DF = pd.DataFrame({'Filename':[x['Artist']+"_"+ x['Title'] for x in L_T_List], 
                    "Artist" :[x['Artist'] for x in L_T_List], 
                    "Title"  :[x['Title'] for x in L_T_List],
                    "Tags"   :[x['Tags'] for x in L_T_List],
                    "VectorizedTags":[x['Vectorized_Tags'] for x in L_T_List]})
    DF.to_csv("VectorizeTagTraing_"+str(_tagNum)+'d.csv',header = True)
    with codecs.open('Tag_Dic_'+str(_tagNum)+'d.json','w',encoding = 'utf8') as outfile:
        json.dump(Tag_Index_Dic,outfile)

In [98]:
if __name__ == "__main__" :
    VectorizedTrainingProducer('Lyric_Tag_Dataset.json', 'Tags_Statistic.json', 10)  